In [56]:
from lib.utils import *
X,y = load_eeg_label_pair()

In [109]:
import torch
from torch import nn
class DeepSleepNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = self._cnn(1,250,64,32)
        self.mp1 = nn.MaxPool1d(8,8)
        self.d1 = nn.Dropout1d(p=.5)
        self.c2 = self._cnn(64,6,128,1)
        self.c3 = self._cnn(128,6,128,1)
        self.c4 = self._cnn(128,6,128,1)
        self.mp2 = nn.MaxPool1d(8,8)
        # concat
        self.d2 = nn.Dropout1d(p=.5)
        # res branch
        self.res_fc = nn.Linear(in_features=128,out_features=1024)
        # temporal
        self.lstm1 = nn.LSTM(128,512,bidirectional=True)
        self.d3 = nn.Dropout1d(p=.5)
        self.lstm2 = nn.LSTM(1024,512,bidirectional=True)
        self.d4 = nn.Dropout1d(p=.5)

        self.d5 = nn.Dropout1d(p=.5)


    def _cnn(self,fan_in,filter_size,fan_out,stride):
        return nn.Sequential(
            nn.Conv1d(fan_in,fan_out,filter_size,stride),
            nn.BatchNorm1d(fan_out),
            nn.ReLU()
        )
    def forward(self,x):
        x = x.view(-1,1,5000)
        x = self.c1(x)
        x = self.mp1(x)
        x = self.d1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.mp2(x)
        x = x.squeeze(2)
        res = self.d1(x)
        h,_ = self.lstm1(x)
        x = self.d3(h)
        h,_ = self.lstm2(x)
        x = self.d4(h)
        
        x = x + self.res_fc(res)
        x = self.d5(x)
        return x
model = DeepSleepNet()

In [108]:
x = model(X)

In [106]:
x.shape

torch.Size([8640, 1024])